# Trabalho 1 - Mineração de Texto 2020/1

O trabalho consiste na predição de atraso em vôos (mais ou menos de 15 minutos de atraso)

## IMPORTANDO LIBS E TABELA DE TREINO

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score


In [ ]:
df_train = pd.read_csv('/kaggle/input/mini-flight-delay-prediction/flight_delays_train.csv')
df_train.info()
df_train.head()

## TRATAMENTO DA ENTRADA DO MODELO

Garantindo que temos amostras com todos os dias possíveis e limpando os valores

In [ ]:
assert df_train['Month'].nunique() == 12
assert df_train['DayofMonth'].nunique() == 31 
assert df_train['DayOfWeek'].nunique() == 7

In [ ]:
time_columns = ['Month', 'DayofMonth', 'DayOfWeek']
for col in time_columns:
    df_train[col] = df_train[col].str.replace(r'c-', '').astype(int)
df_train.head()

Convertendo classes categoricas para classes numéricas utilizando o [LabelEnconder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [ ]:
categorical_columns = ['Origin', 'Dest', 'UniqueCarrier']
le = preprocessing.LabelEncoder()

df_train[categorical_columns + ['dep_delayed_15min']] = df_train[categorical_columns + ['dep_delayed_15min']].apply(le.fit_transform)
df_train.head()

## DESENVOLVIMENTO DOS MODELOS DE CLASSIFICAÇÃO

Verificando distribuição das classes

In [ ]:
dist = df_train['dep_delayed_15min'].value_counts()
print(dist) 

Base bem desequilibrada com a maior parte das amostras no classe de vôos não atrasados.

Criação da base que será usada para a escolha do modelo campeão, utilizando o F1-score como critério.

In [ ]:
df_train.columns

In [ ]:
x = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']
y = 'dep_delayed_15min'

x_train, x_test, y_train, y_test = train_test_split(df_train[x], df_train[y], test_size = 0.2, random_state = 42)

print(f"Tamanho da base de treino: {x_train.shape}")
print(f"Tamanho das classes de treino: {y_train.shape}")
print("")
print(f"Tamanho da base de teste: {x_test.shape}")
print(f"Tamanho das classes de teste: {y_test.shape}")

Resultado para a Regressão Logística

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

print(classification_report(y_test, y_pred))

Resultado para a Árvore de Decisão

In [ ]:
dt1 = DecisionTreeClassifier()
dt1.fit(x_train, y_train)
y_pred = dt.predict(x_test)

print(classification_report(y_test, y_pred))

Resultado para o Gradient Boosting

In [ ]:
gb1 = GradientBoostingClassifier()
gb1.fit(x_train, y_train)
y_pred = gb.predict(x_test)

print(classification_report(y_test, y_pred))

Equilibrando a base (mesmo número de amostras para as duas classes possíveis) e retreinando o modelo

In [ ]:
df_new_train = df_train[df_train[y]==0].sample(dist[1], random_state=42).append(df_train[df_train[y]==1])
df_new_train.info()
df_new_train.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_new_train[x], df_new_train[y], test_size = 0.3, random_state = 42)

print('Resultado da Árvore de Decisãopara a nova Base')
dt2 = DecisionTreeClassifier()
dt2.fit(x_train, y_train)
y_pred = dt.predict(x_test)

print(classification_report(y_test, y_pred))

print('Resultado do Gradient Boosting para a nova Base')
gb2 = GradientBoostingClassifier()
gb2.fit(x_train, y_train)
y_pred = gb.predict(x_test)

print(classification_report(y_test, y_pred))

Analisando a correlação das variáveis e testando hipóteses (como as variáveis relacionas a datas afetam o resultado final? existem varíaveis que podem ser removidas por estarem muito correlacionadas? ...) para retreinar o modelo e avaliar resultado

In [ ]:
df_new_train.corr(method ='pearson') 

Variveis pouco correlacionadas, então nenhuma precisa ser removida. Vamos validar a hipótese da remoção das variáveis relacionadas a datas.

In [ ]:
novo_x = ['DepTime', 'UniqueCarrier', 'Distance']

x_train, x_test, y_train, y_test = train_test_split(df_new_train[x], df_new_train[y], test_size = 0.3, random_state = 42)

print('Resultado da Árvore de Decisãopara a nova Base')
dt3 = DecisionTreeClassifier()
dt3.fit(x_train, y_train)
y_pred = dt.predict(x_test)

print(classification_report(y_test, y_pred))

print('Resultado do Gradient Boosting para a nova Base')
gb3 = GradientBoostingClassifier()
gb3.fit(x_train, y_train)
y_pred = gb.predict(x_test)

print(classification_report(y_test, y_pred))

Apesar do resultado da base sem as variáveis de dada ser muito próximo, o melhor F1 score encontrado foi o do Gradient Boosting com a base balanceada e todas as variáveis (*gb2*), com F1 score de 68%.

## ESCORAGEM DO MODELO CAMPEÃO
 
Agora vamos retreinar o modelo com a base de treino completa, classificar a base de teste original e submeter o desafio.

In [ ]:
df_test = pd.read_csv('/kaggle/input/mini-flight-delay-prediction/flight_delays_test.csv')
df_test.info()
df_test.head()

Precisamos fazer o mesmo tratamento da base de treino na base de teste

In [ ]:
for col in time_columns:
    df_test[col] = df_test[col].str.replace(r'c-', '').astype(int)    

df_test[categorical_columns] = df_test[categorical_columns].apply(le.fit_transform)
df_test.head()

Agora pegamos o modelo, treinamos novamente e escoramos a base de teste

In [ ]:
gb2.fit(df_train[x], df_train[y])
y_pred = gb.predict(df_test)

Salvando o resultado

In [ ]:
output = df_test.copy()
output[y] = y_pred

output.info()
output.to_csv('submission.csv', index=False)